In [33]:
pip install -r rag-requirements-1.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [34]:
import fitz  # PyMuPDF
import json 
import os 
import io
import re
#import requests 
import dotenv 
import transformers
import pypdf
import faiss
#import sqlite3

from dotenv import load_dotenv

from openai import OpenAI
from sentence_transformers import util, SentenceTransformer
from transformers import pipeline, BertTokenizer, BertModel

import pandas as pd
import numpy as np
from io import StringIO
from dotenv import load_dotenv
#from operator import itemgetter

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')
nltk.download('punkt')

from llama_index.core.node_parser import SentenceSplitter

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/joannachang/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joannachang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Setting up Python Environment**

## Instructions for setting up your .env file:

1. Create a .env file in the same directory as this notebook

2. Add the following lines to the .env file:

    OPENAI_API_KEY=<your_openai_api_key>

    HF_TOKEN=<your_huggingface_token>

3. Replace the placeholders with your actual keys.

4. Save the file.

5. Restart the kernel to ensure the keys are loaded correctly.

# Load API Keys into the Notebook Environment:

In [35]:
load_dotenv()

openai = os.getenv('OPENAI_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')
print(HF_TOKEN)

YOUR_HF_TOKEN


# Custom Functions for Chunking the CMU Student Handbook & Measuring Computational Cost
(Optional / not required for the homework that you use these functions)

In [ ]:
# Splitting Text into Sentences. These are not used in the homework.
def split_text_into_sentences_v1(text):
    sentences = nltk.sent_tokenize(text) 
    return sentences

def split_text_into_sentences_v2(text):
    sentences = sent_tokenize(text, language='english')  # Default is usually 'english'
    return sentences

In [69]:
# define a function to split the resumes into sentences and assign unique identifiers:

def split_resumes_to_sentences(df, text_column):
    """
    Split the resumes into individual sentences and assign unique identifiers.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the resumes.
        text_column (str): The name of the column containing the resume texts.
        
    Returns:
        pd.DataFrame: A DataFrame with each sentence and its corresponding unique identifier.
    """
    # Initialize an empty list to hold the resulting data
    sentences_list = []
    
    # Iterate through the DataFrame rows
    for idx, row in df.iterrows():
        # Tokenize the resume text into sentences
        sentences = sent_tokenize(row[text_column])
        
        # Append each sentence along with the original index to the list
        for sentence in sentences:
            sentences_list.append((idx, sentence))
    
    # Convert the list to a DataFrame
    sentences_df = pd.DataFrame(sentences_list, columns=['unique_identifier', 'sentence'])
    
    return sentences_df

In [ ]:

def compute_embedding_costs(text, model_name='all-MiniLM-L6-v2', eps=0.6, min_samples=2):
    """
    Computes the computational cost (in terms of execution time) for creating
    sentence embeddings and paraphrase embeddings.

    Parameters:
    - text (str): The input text to be processed.
    - model_name (str): The name of the model to use for embedding.
    - eps (float): The epsilon value for DBSCAN clustering.
    - min_samples (int): The minimum sample count for DBSCAN clustering.

    Returns:
    - A tuple containing the execution times for sentence embeddings and paraphrase-level embeddings.
    """
    model = SentenceTransformer(model_name)
    
    # Sentence Embedding Timing
    start_time = time.time()
    sentences = sent_tokenize(text)
    sentence_embeddings = model.encode(sentences)
    sentence_embedding_time = time.time() - start_time

    # Paraphrase Embedding (Clustering) Timing
    start_clustering_time = time.time()
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine').fit(sentence_embeddings)
    cluster_labels = clustering.labels_
    
    paraphrase_embeddings = []
    for cluster_id in set(cluster_labels):
        if cluster_id == -1:
            continue
        cluster_sentences = np.array(sentences)[cluster_labels == cluster_id]
        paraphrase = ' '.join(cluster_sentences)
        paraphrase_embeddings.append(paraphrase)
    paraphrase_embedding_time = time.time() - start_clustering_time

    return sentence_embedding_time, paraphrase_embedding_time

# Example usage
if __name__ == "__main__":
    text = ("This is a sample text. It has several sentences, meant to showcase "
            "how embeddings are computed. Some of these sentences may be clustered "
            "together, representing paraphrases or semantically similar groups.")

    sent_time, para_time = compute_embedding_costs(text)
    print(f"Sentence Embedding Time: {sent_time:.4f} seconds")
    print(f"Paraphrase Embedding Time: {para_time:.4f} seconds")

# This function was created with GenerativeAI Assistance. 

In [ ]:
def estimate_model_flops(model_name, text):
    """
    Estimate the FLOPs for generating embeddings for a given text using a specified model.

    Parameters:
    - model_name (str): Model identifier from Hugging Face Transformers.
    - text (str): Text to process.

    Returns:
    - FLOPs (int): An estimated number of floating point operations.
    """
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs['input_ids']

    # Hooks for the operations
    def hook_fn_forward(module, input, output):
        # Attempt to access the tensor shape in a safer manner
        input_shape = input[0].size()
        
        # A generalized fallback if shape isn't what's expected
        if len(input_shape) == 2:  # Assuming shape [batch, seq_len] for simplicity
            batch_size, seq_len = input_shape
            # Hypothetical FLOPs calculation: For demonstration, let's assume it's just the product
            flops = batch_size * seq_len
        elif len(input_shape) > 2:  # Assuming more dimensions (e.g., embeddings)
            flops = torch.prod(torch.tensor(input_shape))
        else:
            # In case of unsupported dimensions, set flops to 0 or some placeholder
            flops = 0

        # Storing calculated FLOPs in the module
        if hasattr(module, '__flops__'):
            module.__flops__ += flops
        else:
            module.__flops__ = flops

    def add_hooks_to_model(model, hook_fn):
        """
        Recursively add hook_fn to all the layers of the model.
        """
        total_flops = 0
        for layer in model.children():
            if list(layer.children()):  # if the layer has children, recursively add hooks
                total_flops += add_hooks_to_model(layer, hook_fn)
            else:
                if hasattr(layer, 'weight'):
                    layer.register_forward_hook(hook_fn)
                    layer.__flops__ = 0
        return total_flops

    add_hooks_to_model(model, hook_fn_forward)

    with torch.no_grad():
        _ = model(**inputs)

    total_flops = sum([mod.__flops__ for mod in model.modules() if hasattr(mod, '__flops__')])

    return total_flops

# Example usage
if __name__ == "__main__":
    model_name = "bert-base-uncased"
    text = "This is an example sentence"
    flops = estimate_model_flops(model_name, text)
    print(f"Estimated FLOPs: {flops}")

# This function was created with GenerativeAI Assistance. 

# Load Data into the Notebook Environment:

In [36]:
doc = fitz.open("CMU Student Handbook 2023-24.pdf")
doc

Document('CMU Student Handbook 2023-24.pdf')

# **Homework 2 Assignment**

## **Section A. Experimenting with Vector Store Query Design (50 points)**

In [37]:
# Define a function to split the handbook into sentences and assign unique identifiers:

def split_handbook_to_sentences(df, text_column):
    """
    Split the handbook into individual sentences and assign unique identifiers.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the handbook.
        text_column (str): The name of the column containing the handbook texts.
        
    Returns:
        pd.DataFrame: A DataFrame with each sentence and its corresponding unique identifier.
    """
    # Initialize an empty list to hold the resulting data
    sentences_list = []
    
    # Iterate through the DataFrame rows
    for idx, row in df.iterrows():
        # Tokenize the handbook text into sentences
        sentences = sent_tokenize(row[text_column])
        
        # Append each sentence along with the original index to the list
        for sentence in sentences:
            sentences_list.append((idx, sentence))
    
    # Convert the list to a DataFrame
    sentences_df = pd.DataFrame(sentences_list, columns=['unique_identifier', 'sentence'])
    
    return sentences_df

### **Choose a method to chunk the text data:**

- [Semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)

- [Recursive chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter)

- [Character chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/character_text_splitter)

- [Token chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/split_by_token)

# Sentence Chunking

### Choose a type of chunker:

#### **Chunker Choices**

In [38]:
# Chunker choice #1: (Sentence-level) - We use this chunker for Part A
text_parser = SentenceSplitter(
    chunk_size=1024
)

In [39]:
text_chunks = [] # create an empty list to store the text chunks.
doc_idxs = []    # create an empty list to store unique identifiers for the text chunks.

# split the CMU handbook up into chunks and assign unique identifiers to each chunk:
for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))


In [40]:
# Glance at the text chunks to observe how the chunks look.
text_chunks 

['1 \n \n \n \nThe Word: Student Handbook \n2023-2024',
 '2 \n \n \nWelcome to The Word 2023-2024 \n \nThe Word student handbook contains information and resources to help you create your \nCarnegie Mellon experience and embrace your role as a valued member of our university. \n \nAt Carnegie Mellon, our ambition is that all students will reach their highest potential in the \nareas of intellectual and artistic pursuits, personal well-being, professional development, \nleadership, and contribution to the larger community. \n \nCommunity membership affords many privileges and likewise responsibilities as we all uphold \nthe standards of our university. To ensure you are knowledgeable of the university’s \nexpectations and the process that will be followed if standards are not met, this handbook \narticulates the rights and responsibilities afforded to and expected of each member of our \ncommunity. \n \nWe hope you will take advantage of this handbook as you prepare for your successful 

In [41]:
text_chunk_df = pd.DataFrame(text_chunks) # put the chunks into a pandas dataframe.
text_chunk_df

,0
0,1 \n \n \n \nThe Word: Student Handbook \n2023...
1,2 \n \n \nWelcome to The Word 2023-2024 \n \nT...
2,3 \n \n \nIndex \n \nCarnegie Mellon Vision an...
3,4 \n \n \nCarnegie Mellon Vision and Mission \...
4,5 \n \n \nCarnegie Mellon Code \n \nStudents a...
...,...
292,293 \n \n \nWithholding a Degree \nBecause a d...
293,294 \n \n \n• Child abuse \n• Criminal mischie...
294,295 \n \n \nCase-specific questions regarding ...
295,296 \n \n \n \n \n \n \n \n \n \n \n \n \n \nA...


In [42]:
# Split the CMU handbook into sentences and include a unique identifier for each sentence:
sentences_df = split_handbook_to_sentences(text_chunk_df, 0) 
sentences_df

,unique_identifier,sentence
0,0,1 \n \n \n \nThe Word: Student Handbook \n2023...
1,1,2 \n \n \nWelcome to The Word 2023-2024 \n \nT...
2,1,"At Carnegie Mellon, our ambition is that all s..."
3,1,Community membership affords many privileges a...
4,1,To ensure you are knowledgeable of the univers...
...,...,...
4046,293,Records of community standards proceedings res...
4047,294,295 \n \n \nCase-specific questions regarding ...
4048,294,Student Resources
4049,295,296 \n \n \n \n \n \n \n \n \n \n \n \n \n \nA...


In [43]:
# get the length of the sentences dataframe:
len(sentences_df)

4051

### **Choose an embedding model to use for creating embeddings of the text chunks and create the Embeddings**

In [44]:
# Define a function to create embeddings for the sentences:

model = SentenceTransformer('bert-base-nli-mean-tokens') 
# Here we select to use a Bert model on HuggingFace to create the embeddings.
# bert-base-nli-mean-tokens is a pre-trained model with 768 dimensions that is a good choice for creating embeddings for sentences.

In [45]:
# Create sentence embeddings:
sentence_embeddings = model.encode(sentences_df['sentence'])

# Check the shape of the sentence embeddings:
sentence_embeddings.shape

(4051, 768)

## **Create a FAISS Vector Store**

In [47]:
# Specify the dimensions of the sentence embeddings:
d = sentence_embeddings.shape[1]

# Specify the number of sentences:
nb = len(set(sentences_df)) # sentence_embeddings.shape[0]

# Specify the number of queries: (may not need it)
# nq = 10000 
# np.random.seed(1234) # set a random number to make the process reproducible
# xb = np.random.random((nb, d)).astype('float32')

# Number of clusters to use for the vector store: (may not need it)
# nlist = 100

In [48]:
# Glance at the shape of the sentence embeddings or dimension for the vector store:
d

768

In [49]:
# Create an index for the vector store:
index = faiss.IndexFlatL2(d)

In [50]:
# Add the sentence embeddings to the index:
index.add(sentence_embeddings)

In [51]:
# check the number of vectors in the index:
index.ntotal

4051

In [52]:
# Train the index:
index.train(sentence_embeddings)

index.is_trained  # check if index is now trained

True

### **Construct Query and Perform Search of the Vector Store**

In [53]:
# Define the number of documents to retrieve from the vector store in response to the query:
def query_vector_store(query: str, k: int = 5):
    '''A function to query the vector store and retrieve the most similar documents.
    parameters:
    - query: str, the query to submit to the vector store.
    - k: int, the number of documents to retrieve from the vector store.

    returns:
    - D: numpy array, the distances to the query from the vectors in the index.
    - I: numpy array, the indices of the vectors in the index that are most similar to the query.
    '''
    # Create an embedding for the query:
    query_embedding = model.encode([query])

    # Search the index for the query, using the number of documents to retrieve specified by k:
    D, I = index.search(query_embedding.astype('float32'), k)

    # Retrieve the string data from the 'text' column of the first index in I:
    results = []
    for idx in I[0]:
        results.append(sentences_df['sentence'].iloc[idx])
    
    return results, D[0], I[0]


In [54]:
# Define the query to submit to the vector store:
queries = [
    "What is the policy statement for the academic integrity policy?",
    "What is the policy violation definition for cheating?",
    "What is the policy statement for improper or illegal communications?",
    "What are CMU's quiet hours?",
    "Where are pets allowed on CMU?"
]

# Query the vector store for each query and collect results
k = 5  # k parameter for Part A
all_results_data = []

print("=" * 80)
print("PART A: Executing queries and collecting results (k=5)")
print("=" * 80)

for query_num, query in enumerate(queries, 1):
    print(f"\nQuery {query_num}: {query}")
    
    # Use the defined query_vector_store function
    results, distances, indices = query_vector_store(query, k=k)
    
    # Create one row of data for each response of each query
    for response_num, (result, distance, idx) in enumerate(zip(results, distances, indices), 1):
        all_results_data.append({
            'Section ': 'A',
            'Query #': query_num,
            'Query Text ': query,
            'k parameter': k,
            'Response #': response_num,
            'Response Text': str(result)
        })
        print(f"  Response {response_num} (Index: {idx}, Distance: {distance:.4f})")

# Create DataFrame
results_df = pd.DataFrame(all_results_data)

# Save to Excel file
output_file = 'rag_hw02_partA_results.xlsx'
results_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"\n{'='*80}")
print(f"✅ Complete! Results saved to: {output_file}")
print(f"📊 Total {len(results_df)} rows of data")
print(f"📋 Contains {len(queries)} queries, each with {k} responses")
print(f"{'='*80}")

# Display first few rows preview
print("\nFirst 5 rows preview:")
print(results_df.head())

PART A: Executing queries and collecting results (k=5)

Query 1: What is the policy statement for the academic integrity policy?
  Response 1 (Index: 1658, Distance: 86.3411)
  Response 2 (Index: 1426, Distance: 115.5850)
  Response 3 (Index: 1441, Distance: 132.0148)
  Response 4 (Index: 649, Distance: 133.1080)
  Response 5 (Index: 2072, Distance: 136.3932)

Query 2: What is the policy violation definition for cheating?
  Response 1 (Index: 859, Distance: 108.2417)
  Response 2 (Index: 842, Distance: 114.6191)
  Response 3 (Index: 358, Distance: 125.0742)
  Response 4 (Index: 2344, Distance: 127.2205)
  Response 5 (Index: 2338, Distance: 133.5898)

Query 3: What is the policy statement for improper or illegal communications?
  Response 1 (Index: 842, Distance: 91.9763)
  Response 2 (Index: 2338, Distance: 96.5746)
  Response 3 (Index: 2344, Distance: 97.5439)
  Response 4 (Index: 2322, Distance: 97.8308)
  Response 5 (Index: 253, Distance: 112.8725)

Query 4: What are CMU's quiet hou

In [56]:
%%time 
# measure the time it takes to search the index
# First, create a query embedding
test_query = "What is the policy statement for the academic integrity policy?"
query_embedding = model.encode([test_query])

# Search the index for the query, using the number of documents to retrieve specified by k
D, I = index.search(query_embedding.astype('float32'), k)  # search the index for the query
print(I) # print the indices of the documents that are most similar to the query

[[1658 1426 1441  649 2072]]
CPU times: user 11 ms, sys: 479 ms, total: 490 ms
Wall time: 1.77 s


In [57]:
# Retrieve and print the string data from 'text' column of the first index in I

first_index = I[0] # Get the first index from I

first_row_string = sentences_df['sentence'].iloc[first_index].sum()  # Use iloc to access the row by index

print(first_row_string) # Print the string data

Reports may be submitted using the Academic 
Integrity Reporting Form.The process for 
appealing an academic integrity violation can be found under the Academic Disciplinary 
Actions Index.102 
 
 
Intellectual Property Disputes 
Disputes concerning rights to intellectual property must be resolved according to the 
procedures set forth in the University's Intellectual Property Policy.The Fair Use Factors 
Members of the University community are required by Section 107 to consider and 
balance the following factors to determine if a use qualifies as a fair use.Existing University policies and principles involving appropriate treatment of 
confidential information include: 
• Gramm-Leach-Bliley Act Information Security Program


## Define System Prompt (e.g. context message) to send to LLM (原本ipynb裡的內容, 沒用到)

In [114]:
# define a function to get retrieve the results from the vector store:
def get_sys_message(user_query: str, retrieval_number: int):
    query_embedding = model.encode([user_query])
    D, I = index.search(query_embedding, retrival_number)  # search
    first_index = I[0]  # Get the first index from I
    first_row_string = sentences_df['sentence'].iloc[first_index].sum()
    return first_row_string 

In [115]:
# use the custom function to retrieve the results from the vector store:
get_sys_message(user_query="Which resume has the most software skills listed?", retrieval_number=100)

'using an account found “logged in” \non a cluster machine).• Attempting to “crack” or guess other users’ passwords.If you are in doubt about the \nappropriateness of something that you want to do, contact the Computing Services Help \nCenter at 412-268-HELP, or send mail to it-help@andrew.cmu.edu and ask first.In cases where an \nindividual or department “owns” machines which use password files controlled by \nanother organization (e.g.• Use the Career and Professional Development Center.As used in this policy, terms such as “computing,” \n“computing/communications systems,” “computing resources,” etc., refer to all computers, \ncommunication systems, and peripherals, software, telephones and systems with similar• Intellectual property is created by individuals, or by groups of individuals, who are \nentitled to choose the course of disclosure; academic freedom of individuals is a \nhigher priority than possible financial rewards.Final examinations can either be comprehensive, coverin

In [116]:
# custom function for using an LLM with a RAG retriever:
def rag_openAI_gpt(
    model: str, 
    query: str, 
    retrieval_number: int, 
    llm_prompt: str):
    
    import openai
    from openai import OpenAI
    
    client = OpenAI()
    
    f=get_sys_message(query, retrieval_number)
    
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": f"Instruction: use the information in {f} to answer the user's question."},
        {"role": "user", "content": f"{llm_prompt}"},
        {"role": "assistant", "content": f"{f}"},
        {"role": "user", "content": "What is the answer?"}
    ]
    )
    return response.choices[0].message.content

In [117]:
gpt_3_5_turbo = "gpt-3.5-turbo"
gpt_4 = "gpt-4"
gpt_4_turbo = "gpt-4-0125-preview"
gpt_4o = "gpt-4o"

## Examples for demonstration only: (原本ipynb裡的內容, 沒用到)

In [118]:
rag_openAI_gpt(model=gpt_3_5_turbo, query="Which resume has the most software skills listed?", retrieval_number=20, llm_prompt="Classify the document and return a label based on the document type or class. Make the label specify which occupation the document pertains to")

'Based on the content provided, the document appears to be a policy document related to computing resources, intellectual property, and responsible use of university resources. The label for this document could be "Policy Document for Computing and Intellectual Property Management." This classification pertains to occupations in the field of information technology, data security, and intellectual property management.'

In [119]:
rag_openAI_gpt(model=gpt_4, query="Which resume has the most software skills listed?", retrieval_number=20, llm_prompt="summarize the resume")

"Your request is unclear. It mentions about a summary, guidelines and policies about using computing systems, intellectual property rights, and also about final exams, but it's not formulating a specific question. Could you please provide more context or clarify what specific information you are looking for?"

In [120]:
rag_openAI_gpt(model=gpt_4_turbo, query="Which resume has the most software skills listed?", retrieval_number=20, llm_prompt="summarize the resume")

'The provided text appears to be a mix of guidelines regarding the use of computing resources, intellectual property rights, and examination policies rather than a resume or a coherent passage with a clear question to answer. It discusses appropriate conduct regarding computer security, the importance of consulting with a Computing Services Help Center for clarity on appropriateness of actions, the process for using career and professional development resources, the creation and disclosure of intellectual property, and the nature of final examinations. If you have a specific question about this passage or need information summarized on a particular topic, please clarify so I can assist you better.'

# Homework requirement:

# Section A

## **Query the vector store using these queries**

**Instruction: set the 'k' parameter to 5**

Query 1: What is the policy statement for the academic integrity policy?

Query 2: What is the policy violation definition for cheating?

Query 3: What is the policy statement for improper or illegal communications?

Query 4: What are CMU’s quiet hours?

Query 5: Where are pets allowed on CMU?


### ***query the vector store with the 5 queries above (don't forget to record the responses in your homework submission spreadsheet: see instructions for a link to the spreadsheet!):***

In [121]:
# query the vector store with the 5 queries above (don't forget to record the responses in your homework submission!):

# **Homework Questions:**

**A.I.** 

(i) Describe these distance metrics: Cosine similarity; Euclidean Distance; Dot Product.

(ii) For each of the metrics you defined in (i), describe how the metric is different from the other metrics.

(iii) For each of the metrics you defined in (i), describe one advantage and one disadvantage of using the metric.

 

**A.II.** Copy and paste the results or information retrieved from the vector store in response to each of the queries you submitted to the vector store in the SPREADSHEET TEMPLATE (please see instructions for a link to the spreadsheet template you should copy and use).  


**A.III.** Qualitatively analyze the responses to your queries submitted to the vector store. Did the queries retrieve the information you were expecting to obtain. Why or why not? Why do you think the queries were successful / unsuccessful in retrieving the information you expected or needed? 

# **Section B. Experimenting with Vector Store Embeddings & Query Parameters (50 points)**

1) Choose 1 of the 5 queries provided in Section A, above, and experiment with submitting the query to the vector store by changing the QUERY and RETRIEVAL_NUMBER parameters in the following manner:


*   A) Baseline query (e.g. query), retrieval_number parameter=1.

*   B) Query, retrieval_number parameter  = 3

*   C) Query, retrieval_number parameter  = 5

*   D) Query, retrieval_number parameter  = 10

**In your written homework submission, record the UNIQUE responses/results of each query submitted to the vector store.**

2. Select a different text chunking method (e.g. word, sentence, paragraph) and:
   
- Chunk your text data using the method.
- Create embeddings for the text. 
- Load the embeddings into the vector store. 
- Submit the same query you selected in B.1, above, and submit it to the vector store 6 times (using the different ‘retrieval_number’ parameter settings defined in B.1, above), and record the responses.

**In your written homework submission, record the responses/results of each query submitted to the vector store.**

### **Homework Questions:**

**B.I.** Explain your rationale for selecting the query you choose in B.1. Why did you choose this query vs. the other 4 queries? 

**B.II.** Copy and paste the responses to the queries you submitted to the vector store in the SPREADSHEET TEMPLATE.

**B.III.** Copy and paste the responses to the queries you submitted to the vector store in the SPREADSHEET TEMPLATE. 

**B.IV.** In observing the responses from the vector store to the queries created in B.1., which ‘k’ parameter do you think retrieved the highest quality / most accurate result? Why do you think this parameter was the best to use with the query?

**B.V.** In observing the responses from the vector store to the queries created in B.2., which ‘k’ parameter do you think retrieved the highest quality / most accurate result? Why do you think this parameter was the best to use with the query?

# **BONUS TASKS / QUESTIONS: Define function to call LLM API**

## Please email Sara for the Bonus Task Python Notebook once you've completed your homework assignment